In [3]:
import xml.etree.ElementTree as ET
import html
from bs4 import BeautifulSoup
from dadata import DadataAsync
import requests
import pandas as pd

In [83]:
# среднесписочная занятость
# https://www.nalog.ru/opendata/7707329152-sshr2019/
# after xmllint
root = ET.parse('emp.xml').getroot()
for type_tag in root.findall('Документ/СведССЧР'):
    value = type_tag.get('КолРаб')
    print(value)

1
1
4
5
4
1
2
12
4
1
3
1
1
3
4
1
0
1
17
6
1
1
3
91
1
1
2
3
9
1
1
3
3
3
0
1
5
1
2
1
3
1
1
1
1
11
1
1
0
1
0
0
1
17
1
1
1
0
1
5
26
2
17
1
1
1
50
2
2
2
1
0
1
1
1
1
2
1
8
4
1
1
1
1
1
15
1
32
3
2
2
4
1
0
14
15
1
1
1
2
4
9
1
2
1
1
30
5
1
10
1
6
1
1
1
1
0
1
1
2
1
0
2
1
1
1
1
1
3
1
5
0
1
4
0
2
0
3
1
1
24
5
1
3
1
10
2
6
1
1
1
0
0
1
1
3
0
1
5
1
1
1
1
7
1
7
1
1
2
0
1
1
5
1
2
1
5
3
2
0
8
2
16
1
1
0
0
2
1
1
0
0
8
1
1
1
1
1
1
2
2
3
1
1
1
1
3
2
1
1
1
1
1
0
4
0
4
0
6
0
2
1
1
1
1
5
1
2
5
3
1
1
7
1
2
0
0
4
1
10
73
2
9
3
1
1
1
5
0
2
1
1
0
1
1
5
2
1
10
1
1
7
46
1
3
4
2
1
2
2
1
1
7
12
4
2
1
1
0
4
2
2
1
1
0
5
3
4
1
2
3
0
1
1
1
10
1
1
1
1
3
1
1
1
1
1
6
1
1
0
1
0
1
14
2
1
1
6
1
5
4
1
4
28
3
1
1
2
1
1
1
2
4
1
0
1
1
2
1
0
6
2
2
1
1
1
0
6
12
1
1
1
0
3
1
1
2
1
1
0
3
1
3
2
3
8
1
1
4
1
90
1
0
1
4
4
2
6
1
2
4
4
1
1
3
2
2
0
1
3
6
9
2
1
2
2
0
2
5
3
2
1
1
2
0
1
3
0
1
1
1
2
2
3
0
1
2
1
1
10
51
1
2
1
1
2
1
1
4
0
0
2
2
1
14
1
2
1
4
1
4
22
1
0
1
2
1
1
1
0
1
3
11
1
2
1
1
1
0
1
7
1
1
4
2
1
1
4
3
2
1
23
1
0
0
1
1
16
1
1
0
1
0


In [31]:
token = "061eb15a78536775b629045888b9829c5b6d175c"
dadata = DadataAsync(token)

/home/demyan/.local/lib/python3.8/site-packages/httpx/_client.py:1781: UserWarning: Unclosed <httpx.AsyncClient object at 0x7fb580717f70>. See https://www.python-httpx.org/async/#opening-and-closing-clients for details.
  warnings.warn(


In [49]:
result = await dadata.suggest("party", "сбербанк")
[r for r in result]

In [80]:
df = pd.DataFrame()
url='https://soctrud.primorsky.ru/employer/?AreaFiasOktmo=9594&ShowWithoutVacancies=True&ShowProvidingQuota=True&Sort=13&Grid-pageSize=100&PageSize=10'
def parse_soup(soup):
    dl = []
    for i, tr in enumerate(soup.find_all("tr")):
        d = {}
        fields = [td.text for td in tr.children]
        d['name'], d['cat'], d['address'], d['emp'], d['vac'] = fields
        d['cat'] = d['cat'].replace('\t', '').replace('\n','').replace('\r','')
        dl += [d]
    return pd.DataFrame(dl[1:])
for i in range(1,20):
    r = requests.get(url + f'&Grid-page={i}')
    page = html.unescape(r.content.decode())
    soup = BeautifulSoup(page)
    df = df.append(parse_soup(soup), ignore_index=True)
    df.to_csv('emps.csv', sep='\t')
    
#     for j, td in enumerate(tr.children):
#         print(i, j, td.text)

ValueError: not enough values to unpack (expected 5, got 1)

In [81]:
pd.read_csv('emps.csv', sep='\t')

,Unnamed: 0,name,cat,address,emp,vac
0,0,"Филиал № 1 ООО ""МЕГАКОМ"" В Г.ВЛАДИВОСТОК",Торговля оптовая напитками…,"690039, г Владивосток, ул Русская, д. 46, корп. Б",57,3
1,1,"Филиал ООО ""Риквэст-Сервис""",Стирка и химическая чистка текстильных и мехов...,"690088, г Владивосток, ш Военное, д. 1, корп. А",93,2
2,2,"""Азиатско-Тихоокеанский Банк"" (ПАО)",Денежное посредничество прочее…,"690091, г Владивосток, ул Светланская, д. 56",209,16
3,3,"""ВЛАДИВОСТОКСКИЙ МОЛОЧНЫЙ КОМБИНАТ"" ФИЛИАЛ АО ...",Производство молока (кроме сырого) и молочной ...,"690087, г Владивосток, ул Стрелочная, д. 19",23,0
4,4,"""ВЛАДИВОСТОКСКИЙ"" ФИЛИАЛ ФГУП ""МОСКОВСКОЕ ПРОП...",Производство медицинских инструментов и оборуд...,"690068, г Владивосток, ул Магнитогорская, д. 8",62,3
...,...,...,...,...,...,...
1321,1321,ЧОУ ВМСОШ,Образование дошкольное…,"690041, г Владивосток, ул Горького, д. 10",42,2
1322,1322,"ЧОУ ДОД ""ЦЕНТР СПОРТИВНОЙ ПОДГОТОВКИ ""ОЛИМПИЕЦ""",Образование дополнительное детей и взрослых…,"690025, г Владивосток, ул Таежная, д. 8",37,0
1323,1323,"Чугуевский филиал КГКУ ""Приморское лесничество""",Лесоводство и прочая лесохозяйственная деятель...,"692623, р-н Чугуевский, с Чугуевка, ул Лесная,...",43,4
1324,1324,"ЧУЗ ""КБ ""РЖД-МЕДИЦИНА"" Г. ВЛАДИВОСТОК",Деятельность больничных организаций,"690003, г Владивосток, ул Верхнепортовая, д. 25",475,6


In [82]:
df.emp[1:].replace('н/д', 0).to_numpy(dtype=int).sum()

302721